# Problem:
    
    I am a teacher and every year we have overnight excursions where we have to organize rooms for around 100 students.
    However there are a number of requirements when organise these rooms:

    Boys and girls MUST be in seperate rooms
    Room sizes and amount of rooms will vary. e.g. 
    This year we might have 4x '5 bed' rooms and 7x '4 bed' rooms, however, 
    we may have restrictions on this so all the boys rooms are together etc..
    Each student allocates 3 students they would like to share with,
    and we promise to get them at least 1 of their friends.


# Solution:

## Considerations:
    Every room will contain at least 1 strongly connected component (SCC) with 2 or more elements,
    otherwise it's impossible to have a valid room.
    This SCC serves as central node with possible other nodes pointing towards it in treelike branches 
    It is possible there is more than one SCC in a room
        
    Also, consider the possible forms of SCC. 
    These will often contain cliques of size 2,3 or 4. 
    For example when 4 friends all choose each other, they form a clique of 4.
    This group could be unconnected to the general graph.
        
    In theory this problem could lack a solution. 
    In practice there will be always be many close friends who pick eachother.
    These can be used to seed to trees as needed.
    
## First approach: brute force
    Only feasible for very small problem sizes, 
       
## Second approach: backtracking    
    Fill each room at random using chaining on preferences.
    (this disregards some possibilities as it prevents multiple SCC hubs)
    When a room is full, check if last element if connected to any other
    if so: explore next room
    else: close branch
        
    For all valid solutions, calculate a solution value, this can be used to find the optimal solutions.
        
## Third approach: backtracking corrected
    The previous approach lifts out the chaining, due to the possible existinence of isolated cliques of size 4.
    So chaining preferences should be ignored, and only checked when the room is full.
        
## Fourth approach: graph backtracking
    Each room should contains at least 1 hub. 
    So use this as a seed, than grow tree inside room backwards. 
    
    If during backtracking a room is left unfilled, seed with a new hub if possible. 
    
    
## Fifth approach: graph heuristic
    As the problem size = 100


In [506]:
# choose number of students

number = 10

In [510]:
# python 3.6

from random import sample,randint
from collections import defaultdict

# Third approach: backtracking corrected

In [508]:
'''Prepare data'''

## student and preferences

students = [x for x in range(number)]
student = defaultdict()
for x in range(number):
    student[x] = set(sample([y for y in range(number) if y != x],3))
    
## rooms   

def generate_rooms(size):
    if size < 4:
        return [size]
    else:
        c = randint(2,size//2)
        return [c,*generate_rooms(size-c)]   

## auxilary list for slicing
    
def cumul(rooms):
    cumul_rooms = []
    for ix,x in enumerate(rooms):
        if not ix:
            cumul_rooms.append(x)
        else:
            cumul_rooms.append(x+cumul_rooms[-1])
    return cumul_rooms

rooms = generate_rooms(number)
cumul_rooms = cumul(rooms)

student, rooms

(defaultdict(None,
             {0: {1, 6, 8},
              1: {5, 6, 9},
              2: {1, 8, 9},
              3: {0, 2, 4},
              4: {0, 1, 2},
              5: {7, 8, 9},
              6: {0, 2, 5},
              7: {2, 3, 9},
              8: {0, 1, 7},
              9: {3, 6, 7}}),
 [4, 3, 3])

In [509]:
'''Backtracking with constraints'''
        
def swap(a, i, j):
    a[i], a[j] = a[j], a[i]

def check_room(a, i):

    room_nok = False
    index = cumul_rooms.index(i)
    for student_in_room in a[i-rooms[index]:i]:
        a_student_ok = False
        for friend in student[student_in_room]:
            if friend in a[i-rooms[index]:i]:
                a_student_ok = True
                break
        if not a_student_ok:
            room_nok = True
            break
    return room_nok

   
def permute(a, i, n, solutions = set()):
    
    if i in cumul_rooms:
        if check_room(a, i):
            return
        
        elif i == n: 
            
            # translate the list 'a' into sorted rooms
            show_rooms = []
            for ix,x in enumerate(cumul_rooms):
                if not ix:
                    show_rooms.append(tuple(sorted(a[0:x])))
                else:
                    show_rooms.append(tuple(sorted(a[cumul_rooms[ix-1]:x]))) 
            a_solution = tuple(sorted(show_rooms))
              
            if a_solution not in solutions:
    
                # value of solution, enables choosing best one
                value = 0
                for x in show_rooms:
                    for y in x:
                        for z in student[y]:
                            if z in x:
                                value += 1
                solutions.add(a_solution)
                print(show_rooms,value) 
                
            return solutions
            
    # backtracking
    for j in range(i, n):
        swap(a, i, j)
        permute(a, i+1, n)
        swap(a, i, j)  

permute(students, 0, len(students))
student,rooms

[(0, 3, 4, 6), (2, 7, 9), (1, 5, 8)] 12
[(0, 3, 4, 8), (1, 2, 6), (5, 7, 9)] 12
[(1, 2, 3, 6), (0, 4, 8), (5, 7, 9)] 11
[(1, 2, 4, 6), (0, 3, 8), (5, 7, 9)] 12
[(1, 4, 5, 8), (0, 3, 6), (2, 7, 9)] 11
[(2, 3, 7, 9), (0, 4, 6), (1, 5, 8)] 13
[(2, 4, 7, 9), (1, 5, 8), (0, 3, 6)] 11


(defaultdict(None,
             {0: {1, 6, 8},
              1: {5, 6, 9},
              2: {1, 8, 9},
              3: {0, 2, 4},
              4: {0, 1, 2},
              5: {7, 8, 9},
              6: {0, 2, 5},
              7: {2, 3, 9},
              8: {0, 1, 7},
              9: {3, 6, 7}}),
 [4, 3, 3])